In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'FashionMNIST'
file_name = 'CLIP_FashionMNIST_test'

df = pd.read_csv(f'./{file_name}.csv')

In [2]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,raw_answer,ill_answer,illless_answer,ill_filter_answer,illless_filter_answer
0,FashionMnist_1,Two friends hiking on a mountain during sunset,low quality,1.5,9,7,10,10,7,10
1,FashionMnist_2,A starry night sky with constellations glowing,low quality,1.5,2,8,6,10,6,10
2,FashionMnist_3,A Thanksgiving feast giving thanks and love,low quality,1.5,1,1,1,10,1,10
3,FashionMnist_4,A gecko on a large leaf in a tropical rainforest,low quality,1.5,1,1,10,10,1,10
4,FashionMnist_5,A Hanukkah menorah lighting symbolizing faith,low quality,1.5,6,6,3,10,4,10
...,...,...,...,...,...,...,...,...,...,...
1147,FashionMnist_1148,"Electrified waterfalls cascade, neon rivers re...",low quality,1.5,2,6,10,10,4,10
1148,FashionMnist_1149,A rocky desert landscape with cacti and tumble...,low quality,1.5,2,6,10,10,0,10
1149,FashionMnist_1150,A majestic waterfall plunging into a crystal-c...,low quality,1.5,5,5,10,10,5,10
1150,FashionMnist_1151,"Eerie silence surrounds snowy mountains, glowi...",low quality,1.5,3,1,10,10,4,10


In [3]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


# **Raw**

In [4]:
y_true = []
y_pred = []

In [5]:
y_true = df['label'].apply(lambda x: str(x)).tolist()
y_pred = df[f'raw_answer'].apply(lambda x: str(x)).tolist()

In [6]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.4115
Precision: 0.4713
Recall: 0.4133
F1 score: 0.3754


# **Illusion**

In [7]:
y_true = []
y_pred = []

In [8]:
y_pred_illless = [str(df.at[row, 'illless_answer']) for row in illusionless_indexes]
y_true_illless = ['10' for i in range(len(y_pred_illless))]

In [9]:
y_true_ill = df['label'].apply(lambda x: str(x)).tolist()
y_pred_ill = df[f'ill_answer'].apply(lambda x: str(x)).tolist()

In [10]:
y_true = y_true_ill + y_true_illless
y_pred = y_pred_ill + y_pred_illless

In [11]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.1389
Precision: 0.2799
Recall: 0.1392
F1 score: 0.092


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# **Filtered**

In [12]:
y_true = []
y_pred = []

In [13]:
y_pred_illless = [str(df.at[row, 'illless_filter_answer']) for row in illusionless_indexes]
y_true_illless = ['10' for i in range(len(y_pred_illless))]

In [14]:
y_true_ill = df['label'].apply(lambda x: str(x)).tolist()
y_pred_ill = df[f'ill_filter_answer'].apply(lambda x: str(x)).tolist()

In [15]:
y_true = y_true_ill + y_true_illless
y_pred = y_pred_ill + y_pred_illless

In [16]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.4475
Precision: 0.505
Recall: 0.4474
F1 score: 0.4128
